In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import random

import matplotlib.pyplot as plt

import emphases

/home/mrm5248/conda/envs/env/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Get stems to annotate
speakers = [str(s) for s in emphases.data.download.LIBRITTS_SPEAKERS]
stems = sorted([
    file.stem for file in (emphases.CACHE_DIR / 'libritts' / 'audio').glob('*.wav')
    if file.stem.split('_')[0] in speakers])
random.seed(emphases.RANDOM_SEED)
random.shuffle(stems)

In [ ]:
eighth = len(stems) // 8

# Initialize annotation progress monitor
monitor = {}
for stem in stems[:eighth]:
    monitor[stem] = [0, 8]
for stem in stems[eighth:2 * eighth]:
    monitor[stem] = [0, 4]
for stem in stems[2 * eighth:4 * eighth]:
    monitor[stem] = [0, 2]
for stem in stems[4 * eighth:]:
    monitor[stem] = [0, 1]

In [ ]:
# Update counts with current progress
cache_directory = emphases.CACHE_DIR / 'annotate'
with open(cache_directory / 'counts.json') as file:
    counts = json.load(file)
for stem, count in counts.items():
    monitor[stem][0] += count

In [ ]:
# Save progress
with open('monitor-annotation.json', 'w') as file:
    json.dump(monitor, file)

In [ ]:
num, den = (
    sum(min(m[0], m[1]) for m in monitor.values()),
    sum(m[1] for m in monitor.values()))
print(num, den, num / den)

In [ ]:
total = len(monitor)
unfinished = sum(m[0] < m[1] for m in monitor.values())
print(unfinished, total, unfinished / total)

In [ ]:
sum(m[0] >= 8 for m in monitor.values())

In [ ]:
sum(m[0] > m[1] for m in monitor.values())

In [ ]:
import torchaudio

files = sorted([
    file for file in (emphases.CACHE_DIR / 'libritts' / 'audio').glob('*.wav')
    if file.stem.split('_')[0] in speakers])

duration = 0.
for file in files:
    info = torchaudio.info(file)
    duration += info.num_frames / info.sample_rate
duration / 3600.

In [ ]:
import pypar

files = sorted([
    file for file in (emphases.CACHE_DIR / 'libritts' / 'alignment').glob('*.TextGrid')
    if file.stem.split('_')[0] in speakers])

words = 0
for file in files:
    alignment = pypar.Alignment(file)
    words += len([word for word in alignment if str(word) != pypar.SILENCE])

print(words)

In [ ]:
with open(emphases.CACHE_DIR / 'annotate' / 'annotations.json') as file:
    annotations = json.load(file)

In [ ]:
len(annotations)

In [ ]:
lengths = [len(a['annotations']) for a in annotations.values()]
plt.hist(lengths)

In [ ]:
max(lengths)